In [1]:
import pandas as pd
import numpy as np
import copy
import requests
from bs4 import BeautifulSoup

In [11]:
import os
import requests
import urllib
import math
import copy
import pandas as pd	
import numpy as np
from bs4 import BeautifulSoup 

class html_tables(object):
    
    def __init__(self, url):
        
        self.url      = url
        self.r        = requests.get(self.url)
        self.url_soup = BeautifulSoup(self.r.text,"lxml")
        
    def read(self):
        
        self.tables      = []
        self.tables_html = self.url_soup.find_all("table")
        
        # Parse each table
        for n in range(0, len(self.tables_html)):
            
            n_cols = 0
            n_rows = 0
            
            for row in self.tables_html[n].find_all("tr"):
                col_tags = row.find_all(["td", "th"])
                if len(col_tags) > 0:
                    n_rows += 1
                    if len(col_tags) > n_cols:
                        n_cols = len(col_tags)
            
            # Create dataframe
            df = pd.DataFrame(index = range(0, n_rows), columns = range(0, n_cols))
            
            # Create list to store rowspan values 
            skip_index = [0 for i in range(0, n_cols)]
            
            # Start by iterating over each row in this table...
            row_counter = 0
            for row in self.tables_html[n].find_all("tr"):
                
                # Skip row if it's blank
                if len(row.find_all(["td", "th"])) == 0:
                    next
                
                else:
                    
                    # Get all cells containing data in this row
                    columns = row.find_all(["td", "th"])
                    col_dim = []
                    row_dim = []
                    col_dim_counter = -1
                    row_dim_counter = -1
                    col_counter = -1
                    this_skip_index = copy.deepcopy(skip_index)
                    
                    for col in columns:
                        
                        # Determine cell dimensions
                        colspan = col.get("colspan")
                        if colspan is None:
                            col_dim.append(1)
                        else:
                            col_dim.append(int(colspan))
                        col_dim_counter += 1
                            
                        rowspan = col.get("rowspan")
                        if rowspan is None:
                            row_dim.append(1)
                        else:
                            row_dim.append(int(rowspan))
                        row_dim_counter += 1
                            
                        # Adjust column counter
                        if col_counter == -1:
                            col_counter = 0  
                        else:
                            col_counter = col_counter + col_dim[col_dim_counter - 1]
                            
                        while skip_index[col_counter] > 0:
                            col_counter += 1

                        # Get cell contents  
                        cell_data = col.get_text()
                        
                        # Insert data into cell
                        df.iat[row_counter, col_counter] = cell_data

                        # Record column skipping index
                        if row_dim[row_dim_counter] > 1:
                            this_skip_index[col_counter] = row_dim[row_dim_counter]
                
                # Adjust row counter 
                row_counter += 1
                
                # Adjust column skipping index
                skip_index = [i - 1 if i > 0 else i for i in this_skip_index]

            # Append dataframe to list of tables
            self.tables.append(df)
        
        return(self.tables)

In [12]:
url =  "https://www.fantasypros.com/nfl/reports/leaders/qb.php?year=2015"
ssa = html_tables(url)
table1 = ssa.read()[0]
table1.head()

,0,1,2,3,4,5
0,Rank,Player,Team,Points,Games,Avg
1,1,Cam Newton,CAR,389.1,16,24.3
2,2,Tom Brady,NE,343.7,16,21.5
3,3,Russell Wilson,SEA,336.4,16,21.0
4,4,Blake Bortles,JAC,316.1,16,19.8


In [17]:
url =  "https://www.pro-football-reference.com/boxscores/201612110ram.htm"
ssa = html_tables(url)
tables = ssa.read()
print(len(tables))
table1 = ssa.read()[1]
table1.head()

2


,0,1,2,3,4,5
0,Quarter,Time,Tm,Detail,ATL,LAR
1,1,14:50,Falcons,Justin Hardy 3 yard pass from Matt Ryan (Matt ...,7,0
2,2,14:53,Falcons,Tevin Coleman 6 yard pass from Matt Ryan (Matt...,14,0
3,,8:54,Falcons,Deion Jones 33 yard interception return (Matt ...,21,0
4,3,13:18,Falcons,Taylor Gabriel 64 yard pass from Matt Ryan (Ma...,28,0


In [2]:
teams_long = [
    'Atlanta Falcons', 'New Orleans Saints', 'Carolina Panthers','Tampa Bay Buccaneers',
    'New York Giants', 'Dallas Cowboys', 'Philadelphia Eagles', 'Washington Redskins',
    'Minnesota Vikings', 'Green Bay Packers', 'Detroit Lions', 'Chicago Bears',
    'Seattle Seahawks', 'San Francisco 49ers', 'Arizona Cardinals',
    'St. Louis Rams', 'Los Angeles Rams',
    'Pittsburgh Steelers', 'Cleveland Browns', 'Cincinnati Bengals', 'Baltimore Ravens',
    'Buffalo Bills', 'New York Jets', 'Miami Dolphins', 'New England Patriots',
    'Tennessee Titans', 'Jacksonville Jaguars', 'Indianapolis Colts', 'Houston Texans',
    'Houston Oilers', 'Tennessee Oilers',
    'Denver Broncos', 'Oakland Raiders', 'San Diego Chargers', 'Kansas City Chiefs'
]

long_to_short = {
    'Atlanta Falcons':'ATL', 'New Orleans Saints':'NOR', 
    'Carolina Panthers':'CAR', 'Tampa Bay Buccaneers':'TAM',
    'New York Giants':'NYG', 'Dallas Cowboys':'DAL',
    'Philadelphia Eagles':'PHI', 'Washington Redskins':'WAS',
    'Minnesota Vikings':'MIN', 'Green Bay Packers':'GNB',
    'Detroit Lions':'DET', 'Chicago Bears':'CHI',
    'Seattle Seahawks':'SEA', 'San Francisco 49ers':'SFO',
    'Arizona Cardinals':'ARI', 'St. Louis Rams':'STL', 'Los Angeles Rams':'LAR',
    'Pittsburgh Steelers':'PIT', 'Cleveland Browns':'CLE',
    'Baltimore Ravens':'BAL', 'Cincinnati Bengals':'CIN',
    'Buffalo Bills':'BUF', 'New York Jets':'NYJ',
    'Miami Dolphins':'MIA', 'New England Patriots':'NWE',
    'Tennessee Titans':'TEN', 'Tennessee Oilers':'TEN', 'Jacksonville Jaguars':'JAX',
    'Indianapolis Colts':'IND', 'Houston Texans':'HOU',
    'Denver Broncos':'DEN', 'Oakland Raiders':'OAK', 
    'Las Vegas Raiders':'LVR?', 'Los Angeles Chargers':'SDG',
    'San Diego Chargers':'SDG', 'Kansas City Chiefs':'KAN'
}

teams_short = [
    'atl','nor','car','tam','nyg','dal','phi','was',
    'min','gnb','det','chi','sea','sfo','crd','ram',
    'pit','cle','cin','rav','buf','nyj','mia','nwe',
    'oti','jax','clt','htx','den','rai','sdg','kan'
]

month = {
    'January':'01','February':'02','March':'03','April':'04','May':'05','June':'06',
    'July':'07','August':'08','September':'09','October':'10','November':'11','December':'12'
}

place_to_team = {
    'TAM':'TAM', 'CAR':'CAR', 'NOR':'NOR', 'ATL':'ATL',
    'NYG':'NYG', 'DAL':'DAL', 'PHI':'PHI', 'WAS':'WAS',
    'MIN':'MIN', 'GNB':'GNB', 'DET':'DET', 'CHI':'CHI',
    'SEA':'SEA', 'SFO':'SFO', 'ARI':'CRD', 'LAR':'RAM', 'STL':'RAM',
    'PIT':'PIT', 'CLE':'CLE', 'CIN':'CIN', 'BAL':'RAV',
    'BUF':'BUF', 'NYJ':'NYJ', 'MIA':'MIA', 'NWE':'NWE',
    'TEN':'OTI', 'JAX':'JAX', 'IND':'CLT', 'HOU':'HTX',
    'OAK':'RAI', 'DEN':'DEN', 'SDG':'SDG', 'KAN':'KAN'
}

In [3]:
def make_soup(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'lxml')
    return soup

In [4]:
def read_header(txt):
    """Tries to read a header from a given input string of HTML text
    
    Returns a tuple consisting of a list of column titles as well as
    the home and away teams"""
    
    header = []
    home_team = 0
    away_team = 0

    # Cycle through each chunk in the raw HTML text
    
    # Look at each section of txt
    for section in txt:
        # For each chunk in that section
        for chunk in section.split():
            
            # Check whether this is the name of a statistic
            # If so, add it to the header
            if "data-stat" in chunk:
                header.append( chunk.split("=")[1].strip('"') )
                
            # This may be home/away team info
            if "</th>" in chunk:
                # strip html tags from the chunk
                chars = chunk.replace("</th","").strip(">")
                
                if "pbp_score_aw" in header[len(header)-1]:
                    away_team = chars                
                elif "pbp_score_hm" in header[len(header)-1]:
                    home_team = chars
                    
            # Or maybe we've reached the end of the header
            elif "</thead>" in chunk:
                break
            elif "</tr>" in chunk:
                break
                
                
    return header, home_team, away_team

In [5]:
def read_table(soup, table):
    """
    Reads a given soup object to find a play-by-play table.
    
    Returns a pandas dataFrame containing the table contents.
    """
    # Differentiate if we're looking for play-by-play table or the spread
    if table == "pbp":
        tablename = "all_pbp"
    elif table == "gameinfo":
        tablename = "all_game_info"
    else:
        tablename = table
        
    
    # Extract the HTML for the "all_pbp" table
    table = soup.find(
        'div', {
            'id': tablename
        })
    
    # If there's no text here, just return 0
    try:
        txt = table.findAll(text=True)
    except:
        return 0
    
    
    # Read the table header directly
    header, home_team, away_team = read_header(txt)
    
    # Turn rows back into strings
    rows = []
    table_body = False
    rowstring = ""
    for section in txt:
        for i, chunk in enumerate(section.split()):
            if ("<tbody>" in chunk) or ("<tr" in chunk):
                table_body = True
            if table_body:
                rowstring += " "+chunk
                if "</tr" in chunk:
                    rows.append(rowstring)
                    rowstring = ""
            if ("</tbody>" in chunk) or ("</table>" in chunk):
                break
                
    # Find data from spaces between tags in each of the rows
    all_entries = []
    stat_names = []
    players_involved = []
    change_of_possession = []
    scoring_plays = []
    in_tag = False
    en_str = ""
    tag_str = ""
    for r, row_string in enumerate(rows):
    
        # Tag-ify the row
        row_en = []
        row_tags = []
        for i, c in enumerate(row_string):
            if c == '<':
                in_tag = True
                row_en.append(en_str)
                en_str = ""
            if not in_tag:
                en_str += c
            elif in_tag:
                tag_str += c
            if c == '>':
                in_tag = False
                row_tags.append(tag_str)
                tag_str = ""
            
        # Make a list of entries in the row
        row_entries = []
        for i, tag in enumerate(row_tags):
            entry = ""
            if ('<th' in tag or '<td' in tag):
                finished = False
                while not finished:
                    for j in range(i,len(row_tags)):
                        entry += row_en[j]
                        if ('/th>' in row_tags[j] or '/td' in row_tags[j]):
                            finished = True
                            break
                row_entries.append(entry)
            
        # Add these entries to the list of all entries
        all_entries.append(row_entries)
            
        # Make a list of players involved in the play
        names = []
        for i, tag in enumerate(row_tags):
            if 'href' in tag:
                names.append(row_en[i+1])
        players_involved.append(names)
        
        # Check for whether play has a change of possession
        change_of_possession.append(0)
        if 'class="divider"' in row_string:
            # Previous play had a change of possession
            change_of_possession[r-1] = 1
            
        # Check for scoring plays
        if 'class=" score"' in row_string:
            scoring_plays.append(1)
        else:
            scoring_plays.append(0)
            
        
        # Make a new header to compare with one from above
        if r == 2:  #this should work but may need to change eventually
            for tag in row_tags:
                for term in tag.split():
                    if "data-stat" in term:
                        stat_names.append( term.split("=")[1].strip('"') )
                  
                
    if tablename == "all_pbp":
        # Add column for players involved
        header.append("players_involved")
        header.append("scoring_play")
        header.append("change_of_poss")
        stat_names.append("players_involved")
        stat_names.append("scoring_play")
        stat_names.append("change_of_poss")
        for i, play in enumerate(all_entries):
            play.append(", ".join(players_involved[i]))
            play.append(scoring_plays[i])
            play.append(change_of_possession[i])
            
        
    # Make a pandas DataFrame from the header and entries
    try:
        df = pd.DataFrame( all_entries, columns=header )
    except:
        df = pd.DataFrame( all_entries, columns=stat_names )
        
    
    # Add home and away team to dataframe
    if type(home_team) is not int:
        df['home_team'] = home_team
        df['away_team'] = away_team
    
        
    return df

In [6]:
def boxscore_url(url):
    base_url = "http://www.pro-football-reference.com/boxscores"
    whole_url = "{0}/{1}.htm".format(base_url,url)
    return whole_url

In [7]:
pbp_soup = make_soup(boxscore_url("201611270rav"))
pbp_df = read_table(pbp_soup,"pbp")
pbp_df.head(5)

,quarter,qtr_time_remain,down,yds_to_go,location,detail,pbp_score_aw,pbp_score_hm,exp_pts_before,exp_pts_after,home_wp,players_involved,scoring_play,change_of_poss,home_team,away_team
0,Quarter,Time,Down,ToGo,Location,Detail,CIN,BAL,EPB,EPA,Win%,,0.0,0.0,BAL,CIN
1,1st Quarter,,0,0,None,None,None,None,None,None,None,None,NaN,NaN,BAL,CIN
2,1,15:00,,,CIN 35,"Mike Nugent kicks off 65 yards, touchback",0,0,0.000,-0.610,62.0,Mike Nugent,0.0,1.0,BAL,CIN
3,1,14:39,1,10,RAV 25,Terrance West right guard for 5 yards (tackle ...,0,0,0.610,0.740,62.3,"Terrance West, Michael Johnson",0.0,0.0,BAL,CIN
4,1,14:38,2,5,RAV 30,Joe Flacco pass complete short right to Terran...,0,0,0.740,1.330,64.0,"Joe Flacco, Terrance West, Vontaze Burfict, Da...",0.0,0.0,BAL,CIN


In [8]:
spread_df = read_table(pbp_soup,"gameinfo")
sp_df = spread_df[ spread_df['info'].str.contains("Vegas") ]
print(sp_df.values[0,1])
spread = sp_df.values[0,1]

Baltimore Ravens -3.5


In [9]:
# Fix missing values in 'scoring_play' and 'change_of_poss' by replacing NaN with 0.
pbp_df.scoring_play.fillna(0, inplace=True)
pbp_df.change_of_poss.fillna(0, inplace=True)
pbp_df.head(3)

,quarter,qtr_time_remain,down,yds_to_go,location,detail,pbp_score_aw,pbp_score_hm,exp_pts_before,exp_pts_after,home_wp,players_involved,scoring_play,change_of_poss,home_team,away_team
0,Quarter,Time,Down,ToGo,Location,Detail,CIN,BAL,EPB,EPA,Win%,,0.0,0.0,BAL,CIN
1,1st Quarter,,0,0,None,None,None,None,None,None,None,None,0.0,0.0,BAL,CIN
2,1,15:00,,,CIN 35,"Mike Nugent kicks off 65 yards, touchback",0,0,0.000,-0.610,62.0,Mike Nugent,0.0,1.0,BAL,CIN


In [10]:
# Figure out where 2nd half starts
#print(pbp_df[ pbp_df.quarter.str.contains("3rd Quarter") ])
halftime_idx = [str(i) for i in pbp_df['quarter']].index("3rd Quarter")
print("Index of row for start of 3rd quarter")
print(halftime_idx)

# Extract first half and second half
first_half = pbp_df.loc[:halftime_idx-2,:]
print("End of first half DataFrame")
print(first_half.tail(3))
second_half = pbp_df.loc[halftime_idx-1:,:]
print("Beginning of second half DataFrame")
print(second_half.head(3))

Index of row for start of 3rd quarter
99
End of first half DataFrame
   quarter qtr_time_remain down yds_to_go location  \
95       2            0:02    3         5   CIN 36   
96       2            0:02                           
97       2            0:02    4         5   CIN 36   

                                   detail pbp_score_aw pbp_score_hm  \
95             Joe Flacco spiked the ball            3           13   
96       Timeout #2 by Cincinnati Bengals            3           13   
97  Justin Tucker 54 yard field goal good            3           16   

   exp_pts_before exp_pts_after home_wp players_involved  scoring_play  \
95          2.280         0.880    90.8       Joe Flacco           0.0   
96          0.000         0.000                                    0.0   
97          0.880         3.000    93.9    Justin Tucker           1.0   

    change_of_poss home_team away_team  
95             0.0       BAL       CIN  
96             0.0       BAL       CIN  
97       

In [11]:
# Make a list of which team has possession to start the half
tmp_list = ["None","None"]
home_away = pbp_df.loc[0,['home_team','away_team']].values
print(home_away)
kickoff_yd = first_half.loc[2,['location']].values
if str(home_away[0]) in str(kickoff_yd):
    # Then home team kicks off
    tmp_list.append(home_away[0])
elif str(home_away[1]) in str(kickoff_yd):
    tmp_list.append(home_away[1])
print(tmp_list)
poss_change = first_half.change_of_poss.values

for i in range(3,len(poss_change)):
#    print(i)
#    print(poss_change[i])
    if poss_change[i] == 0:
        if tmp_list[i-1] == home_away[0]:
            tmp_list.append(home_away[0])
        else:
            tmp_list.append(home_away[1])
    elif poss_change[i] == 1:
        if tmp_list[i-1] == home_away[0]:
            tmp_list.append(home_away[1])
        else:
            tmp_list.append(home_away[0])
#    print(tmp_list)
                
print(tmp_list)

['BAL' 'CIN']
['None', 'None', 'CIN']
['None', 'None', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN']


In [31]:
def offense_tm(half_df):
    # Takes a pbp_df for a particular half.
    # Assuming kickoff is third row in the df, project forward using possession changes.
    # Check each row for a kickoff. This way we can minimize errors caused by weird plays.
    tmp_list = ["None","None"]
    home_away = half_df[['home_team','away_team']].head(1).values[0]
    
    # Get possession for opening kickoff in way that works for either half
    kickoff_yd = half_df.location.head(3).tail(1).values[0]

    # Kickoff is from own half, so 
    if place_to_team[str(home_away[0])] in str(kickoff_yd):
        tmp_list.append(home_away[0])
    elif place_to_team[str(home_away[1])] in str(kickoff_yd):
        tmp_list.append(home_away[1])

    # Make list of possession changes to keep track of
    poss_change = half_df.change_of_poss.values
    
    # Make list of kickoffs to reset with
    kickoffs = half_df['detail'].str.contains("kicks off").values
    kickoffs = [i if i is not None else False for i in kickoffs]
    # Also will need to keep track of field position during kickoff
    fieldpos = half_df.location.values

    
    # Loop through plays of each half
    for i in range(3,len(poss_change)):
        # First, check for a kick-off, same as before
        fieldpos = half_df.location.head(i+1).tail(1).values[0] # need i+1 because head doesn't start at 0
#        print(str(kickoffs[i])+" "+str(fieldpos)+" "+str(half_df.detail.head(i+1).tail(1).values[0]))
        
#        print("i = "+str(i)+" and tmp_list[i-1] = "+str(tmp_list[i-1]))
        # If play is a kickoff from a team's own side
        if (kickoffs[i]) and ('50' not in str(fieldpos)):
            if place_to_team[str(home_away[0])] in str(fieldpos):
                tmp_list.append(home_away[0])
            elif place_to_team[str(home_away[1])] in str(fieldpos):
                tmp_list.append(home_away[1])
            
        
        # If not kickoff (or kickoff was from the 50), check for possession change
        elif poss_change[i-1] == 0:
            if tmp_list[i-1] == home_away[0]:
                tmp_list.append(home_away[0])
            else:
                tmp_list.append(home_away[1])
        else:
            if tmp_list[i-1] == home_away[0]:
                tmp_list.append(home_away[1])
            else:
                tmp_list.append(home_away[0])
                
#        print("len(tmp_list) = "+str(len(tmp_list)))
    
    return tmp_list

In [13]:
#print(second_half[['home_team','away_team']].head(1).values[0])
poss_arrow = offense_tm(first_half)+offense_tm(second_half)
print(poss_arrow)

['None', 'None', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'None', 'None', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'CIN', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', 'BAL', '

In [14]:
# So now add a column to the original DF for the possession arrow
pbp_df['offense_tm'] = offense_tm(first_half)+offense_tm(second_half)
pbp_df.head(3)

,quarter,qtr_time_remain,down,yds_to_go,location,detail,pbp_score_aw,pbp_score_hm,exp_pts_before,exp_pts_after,home_wp,players_involved,scoring_play,change_of_poss,home_team,away_team,offense_tm
0,Quarter,Time,Down,ToGo,Location,Detail,CIN,BAL,EPB,EPA,Win%,,0.0,0.0,BAL,CIN,None
1,1st Quarter,,0,0,None,None,None,None,None,None,None,None,0.0,0.0,BAL,CIN,None
2,1,15:00,,,CIN 35,"Mike Nugent kicks off 65 yards, touchback",0,0,0.000,-0.610,62.0,Mike Nugent,0.0,1.0,BAL,CIN,CIN


In [15]:
def get_score_before_plays(first_half,second_half):
    
    # Get scores from previous play in first half
    home_score_pt1 = first_half.pbp_score_hm.shift(1).values
    away_score_pt1 = first_half.pbp_score_aw.shift(1).values
    # And set score to 0-0 for the first play
    home_score_pt1[2] = 0
    away_score_pt1[2] = 0
    
    # Shift scores same way in second half
    home_score_pt2 = second_half.pbp_score_hm.shift(1).values
    away_score_pt2 = second_half.pbp_score_aw.shift(1).values
    # And set score for first play to what it was at end of first half
    home_score_pt2[2] = first_half.pbp_score_hm.values[-1]
    away_score_pt2[2] = first_half.pbp_score_aw.values[-1]
    
    
    # Sum the two arrays to get a full list for the whole game
    home_score = list(home_score_pt1) + list(home_score_pt2)
    away_score = list(away_score_pt1) + list(away_score_pt2)
    
    return home_score, away_score

In [16]:
print(get_score_before_plays(first_half,second_half))

([nan, ' BAL', 0, '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '6', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '7', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', ' BAL', None, '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', '13', nan, ' BAL', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', '16', ' BAL', None, '16', '16', '16', '16', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '19', '1

In [17]:
def get_winner(df):
    """Return whether the home/away team wins the game"""
    last_play = df.tail(2).head(1)
    score_diff = int(last_play.pbp_score_hm) - int(last_play.pbp_score_aw)
    if score_diff > 0:
        return "home"
    elif score_diff < 0:
        return "away"
    else:
        return "tie"

In [18]:
print(get_winner(pbp_df))

home


In [19]:
# Function to keep track of how many timeouts a team has remaining
def timeouts_half(df):
    home_tm = df['home_team'].head(1).values[0]
    away_tm = df['away_team'].head(1).values[0]
    home_to_rem = []
    away_to_rem = []
    # For each play, check the 'detail' column for 'Timeout'
    for i, detail in enumerate(df['detail'].values):
        det = str(detail)
        if 'Timeout' in det:
#            print(det)
            try:
                to_team = long_to_short[" ".join(det.split()[-2:])]
            except:
                to_team = long_to_short[" ".join(det.split()[-3:])]
#            print(to_team)
            
            if to_team == str(home_tm):
                home_to_rem.append( home_to_rem[-1] - 1 )
                away_to_rem.append( away_to_rem[-1]     )
            elif to_team == str(away_tm):
                away_to_rem.append( away_to_rem[-1] - 1 )
                home_to_rem.append( home_to_rem[-1]     )
        elif i >= 1:
            home_to_rem.append( home_to_rem[-1] )
            away_to_rem.append( away_to_rem[-1] )
        else:
            home_to_rem.append(3)
            away_to_rem.append(3)
            
    return home_to_rem, away_to_rem
            
        
home_to_rem, away_to_rem = timeouts_half(first_half)
print("at end of half, Home TO remaining, Away TO remaining")
print(home_to_rem[-1])
print(away_to_rem[-1])

at end of half, Home TO remaining, Away TO remaining
0
1


In [20]:
# Define a single function to do the necessary cleaning for a single game
def clean_game(raw_df):
#    print(raw_df.info())
    
    # Fix missing values in 'scoring_play' and 'change_of_poss' by replacing NaN with 0.
    raw_df.scoring_play.fillna(0, inplace=True)
    raw_df.change_of_poss.fillna(0, inplace=True)
    
    # Figure out where 2nd half starts
    halftime_idx = [str(i) for i in raw_df['quarter']].index("3rd Quarter")

    # Separate DF into first half and second half
    first_half = raw_df.loc[:halftime_idx-2,:]
    second_half = raw_df.loc[halftime_idx-1:,:]
    
    # Make a column for which team is on offense
    raw_df['offense_team'] = offense_tm(first_half)+offense_tm(second_half)
    
    # Make columns for the score pre-play
    raw_df['home_score'], raw_df['away_score'] = get_score_before_plays(first_half,second_half)
    
    # Fix score at beginning of 2nd, 4th quarters and Overtime periods
    # Iterate through rows. For ith row, check if it's detail contains "2nd Quarter" etc.
    # If so, set score for play in row i+2 equal to score in row i-1.
    def fix_quarters(df):
        for i in range(df.scoring_play.count()):
            row = df.loc[i,:]
            for s in ["2nd Quarter","4th Quarter","Overtime"]:
                if s in row.quarter and "End" not in row.quarter:
#                    print(s)
                    df.loc[i+2,'home_score'] = df.loc[i-1,'pbp_score_hm']
                    df.loc[i+2,'away_score'] = df.loc[i-1,'pbp_score_aw']
#                    print("replaced score")
    fix_quarters(raw_df)
    
    # Make columns for timeouts remaining
    home_to_pt1, away_to_pt1 = timeouts_half(first_half)
    home_to_pt2, away_to_pt2 = timeouts_half(second_half)
#    print(raw_df.info())
#    print("Length of first half df, second half df, corresponding lists")
#    print(first_half.scoring_play.count())
#    print(second_half.scoring_play.count())
#    print(len(home_to_pt1))
#    print(len(away_to_pt2))
#    print(len(home_to_pt1 + home_to_pt2))
    raw_df['home_TO_rem'] = home_to_pt1 + home_to_pt2
    raw_df['away_TO_rem'] = away_to_pt1 + away_to_pt2

    # Make a column for the game winner
    raw_df['winner'] = get_winner(raw_df)
    
    return raw_df

In [21]:
# And now write a function to both grab and clean a pbp table for a single game
def grab_and_clean(game_id):
    
    try:
        # Grab info from web
        pbp_soup = make_soup(boxscore_url(game_id))
    
        # Pull play-by-play table from soup
        pbp_df = read_table(pbp_soup,"pbp")
    
        # Grab Vegas spread from soup
        spread_df = read_table(pbp_soup,"gameinfo")
        spread = spread_df[ spread_df['info'].str.contains("Vegas") ].values[0,1]
        pbp_df['spread'] = spread
    
        # Process the play-by-play dataFrame
        final_df = clean_game(pbp_df)
        final_df['box_url'] = game_id
    
        return final_df
    
    except:
        return 0

In [25]:
# Check to see that everything is working properly.
game_df = grab_and_clean("201611270rav")
game_df.loc[200:, ['quarter', 'detail', 'offense_team', 'home_score','away_score',
                     'home_TO_rem', 'away_TO_rem']]

,quarter,detail,offense_team,home_score,away_score,home_TO_rem,away_TO_rem


In [32]:
# Check to see that everything is working properly.
pbp_soup = make_soup(boxscore_url("201301120den"))
    
# Pull play-by-play table from soup
pbp_df = read_table(pbp_soup,"pbp")
pbp_df = clean_game(pbp_df)



pbp_df.loc[65:75, ['quarter', 'detail','location', 'offense_team', 'home_score','away_score',
                     'home_TO_rem', 'away_TO_rem']]

,quarter,detail,location,offense_team,home_score,away_score,home_TO_rem,away_TO_rem
65,2,Ronnie Hillman up the middle for 3 yards (tack...,RAV 49,DEN,14,14,3,3
66,2,Peyton Manning pass complete deep right to Eri...,RAV 46,DEN,14,14,3,3
67,2,Peyton Manning pass complete short right to Kn...,RAV 14,DEN,14,14,3,3
68,2,Matt Prater kicks extra point good,,DEN,20,14,3,3
69,2,"Matt Prater kicks off 61 yards, returned by Ja...",DEN 35,DEN,21,14,3,3
70,2,Joe Flacco pass incomplete short middle intend...,RAV 25,DEN,21,14,3,3
71,2,Ray Rice up the middle for 6 yards (tackle by ...,RAV 25,DEN,21,14,3,3
72,2,Joe Flacco pass incomplete deep left intended ...,RAV 31,DEN,21,14,3,3
73,2,"Sam Koch punts 50 yards, out of bounds",RAV 31,DEN,21,14,3,3
74,2,Knowshon Moreno left tackle for 2 yards (tackl...,DEN 19,BAL,21,14,3,3


In [24]:
pbp_df.tail(5)

,quarter,qtr_time_remain,down,yds_to_go,location,detail,pbp_score_aw,pbp_score_hm,exp_pts_before,exp_pts_after,...,scoring_play,change_of_poss,home_team,away_team,offense_team,home_score,away_score,home_TO_rem,away_TO_rem,winner
231,OT,15:00,1,10,DEN 34,Ray Rice up the middle for 2 yards (tackle by ...,35,35,3.310,3.040,...,0.0,0.0,DEN,BAL,DEN,35,35,1,0,away
232,OT,14:34,2,8,DEN 32,Ray Rice up the middle for 1 yard (tackle by J...,35,35,3.040,2.480,...,0.0,0.0,DEN,BAL,DEN,35,35,1,0,away
233,OT,14:07,3,7,DEN 31,Ray Rice left guard for 2 yards (tackle by Jus...,35,35,2.480,1.540,...,0.0,0.0,DEN,BAL,DEN,35,35,1,0,away
234,OT,13:29,4,5,DEN 29,Justin Tucker 47 yard field goal good,38,35,1.540,3.000,...,1.0,0.0,DEN,BAL,DEN,35,35,1,0,away
235,End of Overtime,,0,0,None,None,None,None,None,None,...,0.0,0.0,DEN,BAL,DEN,35,38,1,0,away


In [33]:
# Pull game urls from file that we made earlier
games_df = pd.read_csv("datasets/gameinfo1994-2016.csv")
games_df.head(3)

,Unnamed: 0,Week,Day,Date,Time,boxscore,result,OT,Rec,homeaway,...,TotYd.1,PassY.1,RushY.1,TO.1,Offense,Defense,Sp. Tms,team,season,box_url
0,0,1,Sun,September 4,1:00PM ET,boxscore,L,OT,0-1,@,...,352.0,203.0,149.0,1.0,15.16,-13.14,-0.71,atl,1994,199409040det
1,1,2,Sun,September 11,1:00PM ET,boxscore,W,NaN,1-1,NaN,...,421.0,281.0,140.0,4.0,119.06,-78.57,-16.41,atl,1994,199409110atl
2,2,3,Sun,September 18,8:00PM ET,boxscore,L,NaN,1-2,NaN,...,427.0,361.0,66.0,4.0,-7.32,-3.55,-7.88,atl,1994,199409180atl


In [34]:
# Extract just the unique urls from that dataFrame
games_2016 = games_df.loc[games_df.season == 2016, ['box_url']]
game_urls = games_2016.box_url.unique()

In [35]:
import time

In [36]:
# Loop over each of the games in game_urls
# Make a dataframe for each and add it to a list of dataFrames
# Then concatenate all of the dataframes into one big table

all_games = []

for url in game_urls[-3:]:
    if 'bye' not in url:
        print("Looking up "+str(url))
        game_df = grab_and_clean(url)
        all_games.append(game_df)
    
    time.sleep(0.5)
    
big_df = pd.concat(all_games)

     

Looking up 201612180sdg
Looking up 201609110kan
Looking up 201701010sdg


In [37]:
cols = ['detail','offense_team','box_url']
print(big_df[cols].sample(frac=0.05))   

                                                detail offense_team  \
124  Derek Carr pass complete deep left to Mychal R...          OAK   
80   Philip Rivers pass incomplete short left inten...          SDG   
74                       Timeout #1 by Oakland Raiders          OAK   
171  Melvin Gordon middle for 2 yards (tackle by Er...          SDG   
72   Andre Williams right guard for 3 yards (tackle...          SDG   
107  Philip Rivers pass complete short right to Tra...          KAN   
99   Kenneth Farrow right tackle for 6 yards (tackl...          SDG   
83   Charcandrick West middle for 13 yards (tackle ...          KAN   
55   Alex Smith pass complete short middle to Travi...          KAN   
53                  Timeout #1 by Los Angeles Chargers          OAK   
140  Philip Rivers pass complete short right to Ken...          SDG   
25   Philip Rivers pass incomplete deep right inten...          SDG   
24                          Alex Smith pass incomplete          KAN   
131  K

In [30]:
big_df[['location','detail','offense_team','box_url']].query('box_url=="201609180rai"').head(125).tail(40)

,location,detail,offense_team,box_url


In [176]:
game_df = grab_and_clean("201609180rai")

[' Location' None 'RAI 35' 'ATL 25' 'ATL 34' 'ATL 42' 'ATL 35' 'ATL 49'
 'RAI 46' 'RAI 39' 'RAI 36' 'RAI 30' 'RAI 35' 'RAI 35' 'RAI 35' 'RAI 6'
 'RAI 13' 'RAI 22' 'RAI 26' 'RAI 27' 'RAI 29' 'ATL 4' 'ATL 4' 'ATL 4'
 'ATL 15' 'ATL 21' 'ATL 21' 'ATL 21' 'RAI 36' 'RAI 41' 'RAI 41' 'RAI 46'
 'RAI 48' 'ATL 49' 'ATL 49' 'ATL 9' 'ATL 20' 'ATL 26' 'ATL 41' 'ATL 47'
 ' Location' None 'RAI 47' 'RAI 27' 'RAI 11' 'RAI 8' 'RAI 5' 'RAI 3'
 'ATL 35' 'RAI 9' 'RAI 19' 'RAI 24' 'RAI 34' 'RAI 35' 'RAI 38' 'ATL 37'
 'ATL 1' 'ATL 1' 'RAI 35' 'ATL 25' 'ATL 38' 'ATL 38' 'RAI 43' 'RAI 43'
 'RAI 43' 'RAI 33' 'RAI 24' 'RAI 22' 'RAI 21' 'RAI 21' 'ATL 35' 'RAI 2'
 'RAI 11' 'RAI 37' 'RAI 43' 'RAI 33' 'RAI 33' 'RAI 33' 'RAI 36' 'RAI 9'
 'RAI 6' 'RAI 3' 'RAI 3' 'RAI 2' 'ATL 35' 'RAI 25' 'RAI 29' 'RAI 34'
 'RAI 38' 'RAI 38' 'RAI 48' 'ATL 48' 'ATL 48' 'ATL 48' 'ATL 40']
Found a kickoff!
48    Matthew Bosher kicks off 68 yards, returned by...
Name: detail, dtype: object
Found a kickoff!
58    Sebastian Janikowski kicks 

In [177]:
game_df[['location','detail','offense_team']].head(120).tail(35)

,location,detail,offense_team
85,RAI 25,Derek Carr pass complete short right to Clive ...,OAK
86,RAI 29,Derek Carr pass complete short right to Latavi...,OAK
87,RAI 34,Derek Carr pass complete short right to Johnny...,OAK
88,RAI 38,Derek Carr pass complete short middle to Latav...,OAK
89,RAI 38,Timeout #2 by Oakland Raiders,OAK
90,RAI 48,Derek Carr pass complete short right to Lee Sm...,OAK
91,ATL 48,Derek Carr pass incomplete short middle intend...,OAK
92,ATL 48,Timeout #2 by Atlanta Falcons,OAK
93,ATL 48,Derek Carr pass complete short right to Seth R...,OAK
94,ATL 40,Sebastian Janikowski 58 yard field goal no good,OAK


In [38]:
# Loop over each of the games in game_urls
# Make a dataframe for each and add it to a list of dataFrames
# Then concatenate all of the dataframes into one big table

all_games = []
unsuccessful_games = []

for url in game_urls[:]:
    if 'bye' not in url:
        print("Looking up "+str(url))
        game_df = grab_and_clean(url)
        if not type(game_df) is int:
            all_games.append(game_df)
        else:
            print("Lookup for "+str(url)+" unsuccessful")
            unsuccessful_games.append(url)
    
    # Sleep so we don't get shut out
    time.sleep(0.5)
    
big_df = pd.concat(all_games)
print(str(unsuccessful_games))

Looking up 201609110atl
Looking up 201609180rai
Looking up 201609260nor
Looking up 201610020atl
Looking up 201610090den
Looking up 201610160sea
Looking up 201610230atl
Looking up 201610300atl
Looking up 201611030tam
Looking up 201611130phi
Looking up 201611270atl
Looking up 201612040atl
Lookup for 201612040atl unsuccessful
Looking up 201612110ram
Lookup for 201612110ram unsuccessful
Looking up 201612180atl
Looking up 201612240car
Looking up 201701010atl
Looking up 201609110nor
Looking up 201609180nyg
Looking up 201610020sdg
Looking up 201610160nor
Looking up 201610230kan
Looking up 201610300nor
Looking up 201611060sfo
Looking up 201611130nor
Looking up 201611170car
Looking up 201611270nor
Looking up 201612040nor
Looking up 201612110tam
Looking up 201612180crd
Looking up 201612240nor
Looking up 201609080den
Looking up 201609180car
Looking up 201609250car
Looking up 201610100car
Looking up 201610300car
Lookup for 201610300car unsuccessful
Looking up 201611060ram
Looking up 201611130car
L

In [39]:
# Check to see that everything is working properly.
pbp_soup = make_soup(boxscore_url("201612110ram"))
    
# Pull play-by-play table from soup
pbp_df = read_table(pbp_soup,"pbp")
pbp_df = clean_game(pbp_df)



pbp_df.loc[80:110, ['quarter', 'detail', 'offense_team', 'home_score','away_score',
                     'home_TO_rem', 'away_TO_rem']]

IndexError: list index out of range

In [44]:
cols = ['detail','offense_team','box_url']
print(big_df[cols].sample(frac=0.025))   

                                                detail offense_team  \
108  Carson Wentz pass incomplete deep left intende...          PHI   
46   Lamar Miller right tackle for 1 yard (tackle b...          HOU   
11                                                 --           BAL   
157           Josh Lambo kicks off 65 yards, touchback          SDG   
23   Two Point Attempt: Ben Roethlisberger pass com...          PIT   
25   Jerick McKinnon middle for 2 yards (tackle by ...          MIN   
92                                              Detail         None   
15   Derek Carr pass complete short middle to Micha...          OAK   
104  Charcandrick West right tackle for 2 yards (ta...          IND   
30   Dak Prescott right end for 1 yard (tackle by J...          DAL   
81   Charcandrick West middle for 2 yards (tackle b...          KAN   
120  Joe Flacco pass incomplete short right intende...          BAL   
123  Blake Bortles pass complete short left to Marq...          JAX   
123  A

In [45]:
# Write big concatenated dataframe to file
big_df.to_csv("datasets/plays2016_raw.csv")

In [46]:
# Double check that the spreads are working fine
big_df['spread'].unique()

array(['Atlanta Falcons -2.5', 'Oakland Raiders -4.0',
       'New Orleans Saints -2.5', 'Carolina Panthers -2.5',
       'Denver Broncos -3.5', 'Seattle Seahawks -7.0',
       'Atlanta Falcons -4.5', 'Atlanta Falcons -3.0',
       'Philadelphia Eagles -2.0', 'Atlanta Falcons -4.0',
       'Atlanta Falcons -5.0', 'Atlanta Falcons -13.5',
       'Atlanta Falcons -7.5', 'New York Giants -3.5',
       'San Diego Chargers -3.5', 'Kansas City Chiefs -7.0',
       'Seattle Seahawks -1.0', 'New Orleans Saints -5.0',
       'New Orleans Saints -3.0', 'Carolina Panthers -3.5',
       'New Orleans Saints -8.0', 'New Orleans Saints -6.5',
       'Tampa Bay Buccaneers -2.0', 'Arizona Cardinals -3.0',
       'Carolina Panthers -3.0', 'Carolina Panthers -12.0',
       'Carolina Panthers -6.0', 'Oakland Raiders -3.5',
       'Seattle Seahawks -8.0', 'Carolina Panthers -1.0',
       'Washington Redskins -7.0', 'Tampa Bay Buccaneers -3.0',
       'Arizona Cardinals -7.5', 'Tampa Bay Buccaneers -3.5',
 

In [207]:
game_df = grab_and_clean("201609220nwe")

### Also grab all of the plays from 2000-2015

In [47]:
# Extract just the unique urls for games from 2000 to 2015
games_2000to2015 = games_df.loc[ (games_df.season >= 2000) & (games_df.season <= 2015) , ['box_url']]
url_list= games_2000to2015.box_url.unique()
print(url_list)

['200009030atl' '200009100den' '200009170car' ..., '201601030kan'
 '201511220sdg' '201512130kan']


In [48]:
unsuccessful_games = ['201611200sfo', '201609220nwe', '201701010den']

for year in range(2000,2016):
    yr_games = games_df.loc[ games_df.season == year, ['box_url'] ]
    url_list = yr_games.box_url.unique()
    
    all_games = []
    
    for url in url_list[:]:
        if 'bye' not in url:
            print("Looking up "+str(url))
            game_df = grab_and_clean(url)
            if not type(game_df) is int:
                all_games.append(game_df)
            else:
                print("Lookup for "+str(url)+" unsuccessful")
                unsuccessful_games.append(url)
        
        # Sleep so we don't get blocked
        time.sleep(0.5)
    
    # Write this year's plays to its own files
    big_df = pd.concat(all_games)
    big_df.to_csv("datasets/plays{0}_raw.csv".format(year))

print(str(unsuccessful_games))

Looking up 200009030atl
Looking up 200009100den
Looking up 200009170car
Looking up 200009240atl
Looking up 200010010phi
Lookup for 200010010phi unsuccessful
Looking up 200010080atl
Looking up 200010150ram
Looking up 200010220atl
Looking up 200010290atl
Looking up 200011050atl
Looking up 200011120det
Looking up 200011190sfo
Lookup for 200011190sfo unsuccessful
Looking up 200011260rai
Lookup for 200011260rai unsuccessful
Looking up 200012030atl
Looking up 200012170nor
Looking up 200012240atl
Looking up 200009030nor
Looking up 200009100sdg
Lookup for 200009100sdg unsuccessful
Looking up 200009170sea
Looking up 200009240nor
Looking up 200010080chi
Lookup for 200010080chi unsuccessful
Looking up 200010150nor
Lookup for 200010150nor unsuccessful
Looking up 200010290crd
Looking up 200011050nor
Lookup for 200011050nor unsuccessful
Looking up 200011120car
Looking up 200011190nor
Looking up 200011260ram
Looking up 200012030nor
Looking up 200012100sfo
Looking up 200012240nor
Looking up 200012300n